In [4]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

import matplotlib.pyplot as plt
import random

import librosa
import math

In [5]:
# path to json
DATA_PATH = "final_dat.json"

In [6]:
def load_data(data_path):

    with open(data_path, "r") as f:
        data = json.load(f)

    # convert lists to numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print("Data succesfully loaded!")

    return  X, y

In [7]:
# load data
X, y = load_data(DATA_PATH)

Data succesfully loaded!


In [10]:
X.shape

(9986, 130, 13)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
# create train, validation and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2)

# add an axis to input sets
X_train = X_train[..., np.newaxis]
X_validation = X_validation[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [14]:
modelV2 = keras.models.load_model("mgr_model.h5")

In [16]:
X_to_predict = X_test[100]
y_to_predict = y_test[100]

In [23]:
for n in range(10):

  i = random.randint(0,len(X_test))
  # pick a sample to predict from the test set
  X_to_predict = X_test[i]
  y_to_predict = y_test[i]

  print("\nReal Genre:", y_to_predict)

  X_to_predict = X_to_predict[np.newaxis, ...]

  prediction = modelV2.predict(X_to_predict)

  # get index with max value
  predicted_index = np.argmax(prediction, axis=1)

  print("Predicted Genre:", int(predicted_index))


Real Genre: 3
1/1 [==============================] - 0s 47ms/step
Predicted Genre: 9

Real Genre: 2
1/1 [==============================] - 0s 49ms/step
Predicted Genre: 0

Real Genre: 3
1/1 [==============================] - 0s 45ms/step
Predicted Genre: 3

Real Genre: 2
1/1 [==============================] - 0s 49ms/step
Predicted Genre: 2

Real Genre: 9
1/1 [==============================] - 0s 57ms/step
Predicted Genre: 1

Real Genre: 6
1/1 [==============================] - 0s 57ms/step
Predicted Genre: 6

Real Genre: 3
1/1 [==============================] - 0s 59ms/step
Predicted Genre: 3

Real Genre: 2
1/1 [==============================] - 0s 37ms/step
Predicted Genre: 2

Real Genre: 0
1/1 [==============================] - 0s 52ms/step
Predicted Genre: 3

Real Genre: 1
1/1 [==============================] - 0s 60ms/step
Predicted Genre: 1


In [27]:
# Audio files pre-processing
def process_input(audio_file, track_duration):

  SAMPLE_RATE = 22050
  NUM_MFCC = 13
  N_FTT=2048
  HOP_LENGTH=512
  TRACK_DURATION = track_duration # measured in seconds
  SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
  NUM_SEGMENTS = 10

  samples_per_segment = int(SAMPLES_PER_TRACK / NUM_SEGMENTS)
  num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / HOP_LENGTH)

  signal, sample_rate = librosa.load(audio_file, sr=SAMPLE_RATE)
  
  for d in range(10):

    # calculate start and finish sample for current segment
    start = samples_per_segment * d
    finish = start + samples_per_segment

    # extract mfcc
    mfcc = librosa.feature.mfcc(y=signal[start:finish], sr=sample_rate, n_mfcc=NUM_MFCC, n_fft=N_FTT, hop_length=HOP_LENGTH)
    mfcc = mfcc.T

    return mfcc


In [49]:
genre_dict = {0:"disco",1:"country",2:"jazz",3:"classical",4:"metal",5:"pop",6:"rock",7:"blues",8:"reggae",9:"hiphop"}    

In [63]:
new_input_mfcc = process_input("gunsR.mp3", 30)

In [64]:
X_to_predict = new_input_mfcc[np.newaxis, ..., np.newaxis]
X_to_predict.shape

(1, 130, 13, 1)

In [65]:
prediction = modelV2.predict(X_to_predict)

# get index with max value
predicted_index = np.argmax(prediction, axis=1)

print("Predicted Genre:", genre_dict[int(predicted_index)])

1/1 [==============================] - 0s 55ms/step
Predicted Genre: rock
